In [1]:
import os
import sys
from datetime import datetime
from time import strftime

sys.path.append("../")
from data_loader import load_data, get_entire_data, get_features

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
_train, _test = load_data()
entire_data = get_entire_data(_train, _test)
df = get_features(entire_data)
df.head()

object
datetime64[ns]
datetime64[ns]
datetime64[ns]


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,userID_answerCode_mean,userID_answerCode_count,userID_answerCode_sum,userID_answerCode_var,...,hour_answerCode_count,hour_answerCode_sum,hour_answerCode_var,hour_answerCode_median,hour_answerCode_Level,month_answerCode_mean,month_answerCode_count,month_answerCode_sum,month_answerCode_var,month_answerCode_median
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0.630872,745,470,0.233185,...,110005,71546,0.227803,1.0,2,0.681427,189115,128868,0.217086,1.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0.630872,745,470,0.233185,...,110005,71546,0.227803,1.0,2,0.681427,189115,128868,0.217086,1.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0.630872,745,470,0.233185,...,110005,71546,0.227803,1.0,2,0.681427,189115,128868,0.217086,1.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0.630872,745,470,0.233185,...,110005,71546,0.227803,1.0,2,0.681427,189115,128868,0.217086,1.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0.630872,745,470,0.233185,...,110005,71546,0.227803,1.0,2,0.681427,189115,128868,0.217086,1.0
